In [1]:
!pip install python-nmap


  Using cached python_nmap-0.7.1-py2.py3-none-any.whl


In [1]:
import nmap

scanner = nmap.PortScanner()
print(scanner.nmap_version())


(7, 95)


In [3]:
import nmap
import xml.etree.ElementTree as ET

class NmapScanner:
    def __init__(self):
        """Initialize the Nmap Port Scanner"""
        self.scanner = nmap.PortScanner()

    def scan(self, hosts="127.0.0.1", ports="1-1000", arguments="-sV"):
        """
        Runs an Nmap scan on the given host(s) and ports.
        
        :param hosts: IP address or range to scan (default is localhost)
        :param ports: Ports to scan (default: 1-1000)
        :param arguments: Additional Nmap arguments (default: -sV for service detection)
        :return: Parsed scan results
        """
        try:
            print(f"Scanning {hosts} on ports {ports} with arguments: {arguments}...")
            self.scanner.scan(hosts=hosts, ports=ports, arguments=arguments)
            
            if not self.scanner.all_hosts():
                raise Exception("No hosts found. Check the IP address and try again.")
            
            return self.parse_scan_results()
        
        except Exception as e:
            return {"error": str(e)}

    def parse_scan_results(self):
        """
        Parses the Nmap scan results and returns structured data.
        :return: Dictionary containing scan results
        """
        results = {}
        for host in self.scanner.all_hosts():
            results[host] = {
                "state": self.scanner[host].state(),
                "protocols": {}
            }
            for proto in self.scanner[host].all_protocols():
                results[host]["protocols"][proto] = {}
                ports = self.scanner[host][proto].keys()
                for port in ports:
                    results[host]["protocols"][proto][port] = {
                        "state": self.scanner[host][proto][port]["state"],
                        "name": self.scanner[host][proto][port]["name"],
                        "product": self.scanner[host][proto][port].get("product", "Unknown"),
                        "version": self.scanner[host][proto][port].get("version", "Unknown")
                    }
        return results


# Example usage
if __name__ == "__main__":
    scanner = NmapScanner()
    result = scanner.scan(hosts="127.0.0.1", ports="80,443", arguments="-sV")
    print(result)


Scanning 127.0.0.1 on ports 80,443 with arguments: -sV...
{'127.0.0.1': {'state': 'up', 'protocols': {'tcp': {80: {'state': 'closed', 'name': 'http', 'product': '', 'version': ''}, 443: {'state': 'closed', 'name': 'https', 'product': '', 'version': ''}}}}}
